# 1-dimensional quadrotor vertical flight -- Stabilization

## 1. Problem formulation
Consider 
$$ x^e_{t+1}=A_* x^e_t + B_* u^e_t + w_t$$
where  $w_t$ is i.i.d disturbances with zero mean. 
Consider quadratic cost:
$$ c(x_t,u_t)=\frac{1}{2} x_t^\top Q x_t + \frac{1}{2} u_t^\top R u_t$$


## 2. Parameter choices


In [1]:
import numpy as np
import random
import sklearn
from sklearn.datasets import make_regression 
import pylab
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import math
import quadprog
import scipy.linalg as la
import control
from numpy.linalg import matrix_power as matp
from numpy.linalg import multi_dot as matdot
from numpy.linalg import eig

import pandas as pd
#from cvxopt import spmatrix, matrix, solvers
import cvxopt
cvxopt.solvers.options['show_progress'] = False        # !!!


pd.options.display.max_columns = None
pd.options.display.max_rows = None


## Useful functions

In [2]:
# define function to return A, B from alpha, beta estimate
def system_matrices(alpha, beta):
    A=np.array([[1, A12 ],[0,1-beta]])
    B=np.array([[0],[alpha]])
    return A, B


# Task 2: compute discrete time LQR controller, u=-Kx, output K
def dlqr(A,B,Q,R):
    """
    Solve the discrete time lqr controller.
    x[k+1] = A x[k] + B u[k]
    cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k]
    compute K 
    optimal controller is u=-Kx   % Notice that when implement it, it has a minus sign!
    """
    # first, solve theR ricatti equation
    P = np.matrix(la.solve_discrete_are(A, B, Q, R))
    # compute the LQR gain
    K =  np.matrix(la.inv(B.T*P*B+R)*(B.T*P*A))
    return K


# Input, A, B, K, output: A-BK
def closed_loop_LTI(A, B, K):
    AK = A-B.dot(K)
    return AK


#print(closed_loop_LTI(np.eye(2),np.array([[0],[1]]),np.array([[0.3,4]])))

## System Dynamics Parameter Choices

In [13]:
alpha_star=1
beta_star=0.25

A12=1

# xe1_min=-5
# xe1_max=5

# xe2_min=-5
# xe2_max=5

K_stab= np.array([[2/3, 1]])
print('Kstab =',K_stab)


# # cost parameters 
# Q=np.eye(2)
# R=0.5

# # disturbance choices
# w_max=0.3

Kstab = [[0.66666667 1.        ]]


In [5]:
# generate true A_star, B_star, system

A_star, B_star = system_matrices(alpha_star, beta_star)
print('Astar =')
print(A_star)
print('---------------------')
print('Bstar =')
print(B_star)


Astar =
[[1.   1.  ]
 [0.   0.75]]
---------------------
Bstar =
[[0]
 [1]]


## Task 1: find $K_{stab}$ to robustly stabilize every (A,B) in the initial uncertainty set.



In [11]:
# test if K_stab stabilize all (A,B) in the range above
## let me do 2 loops, one over A_hat, another over B_hat 


# alpha_min=0.6
# alpha_max =1.6
# beta_min=0.1
# beta_max=0.3


alpha_hat_ranges=np.linspace(alpha_min, alpha_max, num=20)
beta_hat_ranges=np.linspace(beta_min, beta_max, num=10)
maximum_e_value_robust=0
is_looping = True
for alpha_hat in alpha_hat_ranges:
    for beta_hat in beta_hat_ranges:
        A_hat, B_hat = system_matrices(alpha_hat, beta_hat)
        A_K_hat = A_hat - B_hat.dot(K_stab)
        eigVal_hat = eig(A_K_hat)[0]
        max_evalue_hat= np.max(np.abs(eigVal_hat))
        if maximum_e_value_robust < max_evalue_hat:
            maximum_e_value_robust = max_evalue_hat
        if maximum_e_value_robust>1.2:
            print('beta = ', beta_hat)
            print(A_hat)
            print('alpha = ', alpha_hat)
            print(B_hat)
            
            is_looping = False
            break
    if not is_looping:
        break
    

print(maximum_e_value_robust)  # if it is smaller than 1, we have a robustly stabilizing linear controller

# to be more perfect, we would hope for maximum_e_value_robust<=0.6



0.8366600265340756


In [ ]:
0.6**6

In [ ]:
# given A, B, u=-Kx, test the eigenvalue
alpha_hat = alpha_min
beta_hat=beta_min
A_hat, B_hat = system_matrices(alpha_hat, beta_hat)
A_K_hat = A_hat - B_hat.dot(K_stab)  # estimated closed loop system transition matrix
eigVal_hat = eig(A_K_hat)[0] # e-value First output is [0]
max_evalue_hat= np.max(np.abs(eigVal_hat))

print('Ahat =')
print(A_hat)
print('---------------------')
print('Bhat =')
print(B_hat)
print('---------------------')
print('AK_hat =')
print(A_K_hat)
print('---------------------')
print('eigenvalues of AK_hat')
print(eigVal_hat)
print('---------------------')
print('maximum absolute eigenvalues of AK_hat')
print(max_evalue_hat)

## Task 2: Solve discrete-time LQR controller (unconstrained)
solved and added to useful function

## Task 3: for any K, given w's span, and true system, solve its trajectory span, i.e., max x1, min x1, max x2, min x2, max u, min u

### Constraint and disturbance parameters

In [8]:
random.seed(13) # this is a global random seed for random package
np.random.seed(107) # this is a global random seed for np package
# After setting them, no matter what random package we use, we give same output
Q_test= np.eye(2)
R_test=1
print('Current R is ', R_test)
w_max_test=0.2  # use Bernoulli to generate trajectories

#x1_max_test= x1_max_reach-0.05
x1_max_test=5
x1_min_test=-1
print('x1 upper bound: ', x1_max_test)
print('x1 lower bound: ', x1_min_test)
x2_max_test= 2
x2_min_test= -2
print('x2 upper bound: ', x2_max_test)
print('x2 lower bound: ', x2_min_test)
u_max_test=2
u_min_test=-9.8
print('u upper bound: ', u_max_test)
print('u lower bound: ', u_min_test)

Current R is  1
x1 upper bound:  5
x1 lower bound:  -1
x2 upper bound:  2
x2 lower bound:  -2
u upper bound:  2
u lower bound:  -9.8


In [6]:
# Useful functions for Task 3

# estimate the inner approximation of AK's reachable set by x_T
# output: maximum reachable level on x_1,t, and the residual error
def inner_reachable_x1_max(T_reachable, AK, w_max):
    # consider T-step reachable
    x1_max_reach = 0 # Since w is symmetric, x1_min_reach=-x1_max_reach
    for i in range(T_reachable):
        matrixpower_i=matp(AK,i)
        Matrix_interm = np.array([[1,0]]).dot(matrixpower_i)
        current=la.norm(Matrix_interm, np.inf)
        x1_max_reach += current*w_max
    matrixpower_T=matp(AK,T_reachable)
    Matrix_term = np.array([[1,0]]).dot(matrixpower_T)
    residual_error=la.norm(Matrix_term, np.inf)*w_max
    return x1_max_reach, residual_error




# Function: 
# implement u=-Kx for many trials,, Ax+Bu+w, u=-Kx, w is given
# generate trajectories, and upper and lower bound of trajectories
def LTI_trajectories_given_w(T_horizon,N_traj,A, B, K, x1_initial, x2_initial,w1_trajectories,w2_trajectories):
    x1_implement_max = np.zeros(T_horizon)
    x2_implement_max = np.zeros(T_horizon)
    u_implement_max = np.zeros(T_horizon)
    x1_implement_min = np.zeros(T_horizon)
    x2_implement_min = np.zeros(T_horizon)
    u_implement_min = np.zeros(T_horizon)
    x1_trajectories = np.zeros([N_traj,T_horizon])
    x2_trajectories = np.zeros([N_traj,T_horizon])
    u_trajectories = np.zeros([N_traj,T_horizon])
    


    for trial_index in range(N_traj):

        x1_trajectories[trial_index,0]=x1_initial
        x2_trajectories[trial_index,0]=x2_initial
        for stage in range(T_horizon):
            x_current = np.array([[x1_trajectories[trial_index,stage]],[x2_trajectories[trial_index,stage]]])
            u_current = -K.dot(x_current)
            u_trajectories[trial_index,stage]= u_current
            # w_current = np.random.uniform(-w_max_test,w_max_test,2).reshape((2,1)) # uniform dist
            w_current = np.array([[w1_trajectories[trial_index,stage]],[w2_trajectories[trial_index,stage]]])
#             print('Trial is ', trial_index, 'Stage is ', stage)
#             print('current disturbance is', w_current)
           

            x_next= A.dot(x_current)+B.dot(u_current)+w_current
            if stage < T_horizon-1:
                x1_trajectories[trial_index,stage+1]=x_next[0]
                x2_trajectories[trial_index,stage+1]=x_next[1]
            # print(x1_trajectories)

        x1_implement_max = np.maximum(x1_implement_max, x1_trajectories[trial_index,:])
        x2_implement_max = np.maximum(x2_implement_max, x2_trajectories[trial_index,:])
        u_implement_max = np.maximum(u_implement_max, u_trajectories[trial_index,:])
        x1_implement_min = np.minimum(x1_implement_min, x1_trajectories[trial_index,:])
        x2_implement_min = np.minimum(x2_implement_min, x2_trajectories[trial_index,:])
        u_implement_min = np.minimum(u_implement_min, u_trajectories[trial_index,:])
    return x1_implement_max, x1_implement_min, x1_trajectories, \
           x2_implement_max, x2_implement_min, x2_trajectories, \
           u_implement_max, u_implement_min, u_trajectories





In [ ]:
random.seed(13) # this is a global random seed for random package
np.random.seed(107) # this is a global random seed for np package
# After setting them, no matter what random package we use, we give same output
Q_test= np.eye(2)
R_test=1
print('Current R is ', R_test)
w_max_test=0.2

A_test=np.array([[1,1],[0,0.75]])
B_test=np.array([[0],[1]])

K_test= dlqr(A_test,B_test,Q_test,R_test)
print('K_lqr is ', K_test)

AK_test=closed_loop_LTI(A_test, B_test, K_test)
x1_initial = 0
x2_initial = 0
T_horizon=20
N_traj=1
x1_max_reach, residual_error=inner_reachable_x1_max(T_horizon, AK_test, w_max_test)
print('When R=', R_test, ' w=', w_max_test,' maximum altitude: ', x1_max_reach, ' with residual ', residual_error)



# generate w disturbances here, see if it is controlled by random seed?
w1_trajectories = np.zeros([N_traj,T_horizon])
w2_trajectories = np.zeros([N_traj,T_horizon])
w1_trajectories = 2*np.random.binomial(size=N_traj*T_horizon, n=1, p= 0.5).reshape(N_traj,T_horizon)-1  
w1_trajectories = w_max_test*w1_trajectories
w2_trajectories = 2*np.random.binomial(size=N_traj*T_horizon, n=1, p= 0.5).reshape(N_traj,T_horizon)-1  
w2_trajectories = w_max_test*w2_trajectories

print(w1_trajectories)
print(w2_trajectories)

x1_traj_max, x1_traj_min, x1_traj, x2_traj_max, x2_traj_min, x2_traj, u_traj_max, u_traj_min, u_traj \
= LTI_trajectories_given_w(T_horizon,N_traj,A_test, B_test, K_test, x1_initial, x2_initial,w1_trajectories,w2_trajectories)









# plot prep
Plot_Time_index = np.array([i for i in range(T_horizon)])
x1_upperbdd = np.full((1,T_horizon), x1_max_test)[0]
x1_lowerbdd = np.full((1,T_horizon), x1_min_test)[0]



# plot
plt.plot(Plot_Time_index,x1_traj_max)
plt.plot(Plot_Time_index,x1_traj_min)
plt.plot(Plot_Time_index,x1_upperbdd)
plt.plot(Plot_Time_index,x1_lowerbdd)

plt.xlabel('t')
plt.ylabel('altitude error x_{1,t}')
plt.title('LQR controller reachable set')
#plt.legend(['Sine', 'Cosine'])
plt.show()  



plt.plot(Plot_Time_index,x2_traj_max)
plt.plot(Plot_Time_index,x2_traj_min)


plt.xlabel('t')
plt.ylabel('velocity x_{2,t}')
plt.title('R=1, LQR controller reachable set')
#plt.legend(['Sine', 'Cosine'])
plt.show()  


plt.plot(Plot_Time_index,u_traj_max)
plt.plot(Plot_Time_index,u_traj_min)


plt.xlabel('t')
plt.ylabel('Thrust u')
plt.title('R=1, LQR controller reachable set')
#plt.legend(['Sine', 'Cosine'])
plt.show()  



w_upperbdd = np.full((1,T_horizon), w_max_test)[0]
w_lowerbdd = np.full((1,T_horizon), -w_max_test)[0]



plt.plot(Plot_Time_index,w1_trajectories[0,:])
plt.plot(Plot_Time_index,w_upperbdd)
plt.plot(Plot_Time_index,w_lowerbdd)

plt.xlabel('t')
plt.ylabel('disturbance w')
plt.title('realized disturbances')
#plt.legend(['Sine', 'Cosine'])
plt.show()  



In [ ]:
random.seed(13) # this is a global random seed for random package
np.random.seed(2021) # this is a global random seed for np package
# After setting them, no matter what random package we use, we give same output
w1_trajectories = 2*np.random.binomial(size=N_traj*T_horizon, n=1, p= 0.5).reshape(N_traj,T_horizon)-1  
w1_trajectories = w_max_test*w1_trajectories
w2_trajectories = 2*np.random.binomial(size=N_traj*T_horizon, n=1, p= 0.5).reshape(N_traj,T_horizon)-1  
w2_trajectories = w_max_test*w2_trajectories
print(random.random())
print(w1_trajectories)
print(w2_trajectories)

## Task 4: find K_rob_safe and model uncertainty, to be robustly safe

In [31]:
# test if K_tight (LQR with R=0.01) stabilize all (A,B) in the range above
Q_test= np.eye(2)
R_test=0.01
print('Current R is ', R_test)
w_max_test=0.2


alpha_min=0.5
alpha_max =1.2
beta_min=0.2
beta_max=0.4


K_tight= dlqr(A_star,B_star,Q_test,R_test)
print('K_tight is ', K_tight)
## let me do 2 loops, one over A_hat, another over B_hat 
alpha_hat_ranges=np.linspace(alpha_min, alpha_max, num=20)
beta_hat_ranges=np.linspace(beta_min, beta_max, num=10)
maximum_e_value_robust=0
is_looping = True
for alpha_hat in alpha_hat_ranges:
    for beta_hat in beta_hat_ranges:
        A_hat, B_hat = system_matrices(alpha_hat, beta_hat)
        A_K_hat = A_hat - B_hat.dot(K_tight)
        eigVal_hat = eig(A_K_hat)[0]
        max_evalue_hat= np.max(np.abs(eigVal_hat))
        if maximum_e_value_robust < max_evalue_hat:
            maximum_e_value_robust = max_evalue_hat
        if maximum_e_value_robust>1.2:
            print('beta = ', beta_hat)
            print(A_hat)
            print('alpha = ', alpha_hat)
            print(B_hat)
            
            is_looping = False
            break
    if not is_looping:
        break
    

print(maximum_e_value_robust)  # if it is smaller than 1, we have a robustly stabilizing linear controller

# to be more perfect, we would hope for maximum_e_value_robust<=0.6


Current R is  0.01
K_tight is  [[0.61461401 1.36178088]]
0.6530057916597312
